In [71]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [119]:
train = pd.read_csv("train_ready_for_model.csv")
train.head()

,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,...,ApplicantIncome_CoapplicantIncome,ApplicantIncome_Credit_History,ApplicantIncome_LoanAmount,ApplicantIncome_Loan_Amount_Term,CoapplicantIncome_Credit_History,CoapplicantIncome_LoanAmount,CoapplicantIncome_Loan_Amount_Term,Credit_History_LoanAmount,Credit_History_Loan_Amount_Term,LoanAmount_Loan_Amount_Term
0,0,1,1,0,1,0,0,0,1,0,...,0.0,5849.0,777917.0,2105640.0,0.0,0.0,0.0,133.0,360.0,47880.0
1,0,1,0,1,0,1,0,0,1,0,...,6911164.0,4583.0,586624.0,1649880.0,1508.0,193024.0,542880.0,128.0,360.0,46080.0
2,0,1,0,1,1,0,0,0,1,0,...,0.0,3000.0,198000.0,1080000.0,0.0,0.0,0.0,66.0,360.0,23760.0
3,0,1,0,1,1,0,0,0,0,1,...,6090714.0,2583.0,309960.0,929880.0,2358.0,282960.0,848880.0,120.0,360.0,43200.0
4,0,1,1,0,1,0,0,0,1,0,...,0.0,6000.0,846000.0,2160000.0,0.0,0.0,0.0,141.0,360.0,50760.0


In [120]:
test = pd.read_csv("test_ready_for_model.csv")
test.head()

,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,...,ApplicantIncome_CoapplicantIncome,ApplicantIncome_Credit_History,ApplicantIncome_LoanAmount,ApplicantIncome_Loan_Amount_Term,CoapplicantIncome_Credit_History,CoapplicantIncome_LoanAmount,CoapplicantIncome_Loan_Amount_Term,Credit_History_LoanAmount,Credit_History_Loan_Amount_Term,LoanAmount_Loan_Amount_Term
0,0,1,0,1,1,0,0,0,1,0,...,0.0,5720.0,629200.0,2059200.0,0.0,0.0,0.0,110.0,360.0,39600.0
1,0,1,0,1,0,1,0,0,1,0,...,4614000.0,3076.0,387576.0,1107360.0,1500.0,189000.0,540000.0,126.0,360.0,45360.0
2,0,1,0,1,0,0,1,0,1,0,...,9000000.0,5000.0,1040000.0,1800000.0,1800.0,374400.0,648000.0,208.0,360.0,74880.0
3,0,1,0,1,0,0,1,0,1,0,...,5957640.0,2340.0,234000.0,842400.0,2546.0,254600.0,916560.0,100.0,360.0,36000.0
4,0,1,1,0,1,0,0,0,0,1,...,0.0,3276.0,255528.0,1179360.0,0.0,0.0,0.0,78.0,360.0,28080.0


In [121]:
result = test[["Loan_ID"]]
result.head()

,Loan_ID
0,LP001015
1,LP001022
2,LP001031
3,LP001035
4,LP001051


In [75]:
dcc=test.select_dtypes(exclude=[np.number]).columns
dcc

Index(['Loan_ID'], dtype='object')

In [6]:
#Define target and ID columns:
target = 'Loan_Status'
IDcol = ['Loan_ID']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    kfold = model_selection.KFold(n_splits=10, random_state=7)

    #Perform cross-validation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [70]:
predictors=['Property_Area_Rural',
 'Property_Area_Semiurban',
 'Credit_History',
 'Gender_Male_Property_Area_Semiurban',
 'Gender_Male_Credit_History',
 'Married_Yes_Education_Graduate',
 'Married_Yes_Property_Area_Semiurban',
 'Married_Yes_Credit_History',
 'Dependents_0_Property_Area_Semiurban',
 'Dependents_0_Credit_History',
 'Dependents_1_Property_Area_Rural',
 'Dependents_2_Credit_History',
 'Education_Graduate_Property_Area_Semiurban',
 'Education_Graduate_Credit_History',
 'Self_Employed_No_Property_Area_Rural',
 'Self_Employed_No_Property_Area_Semiurban',
 'Self_Employed_No_Credit_History',
 'Self_Employed_Yes_Credit_History',
 'Property_Area_Rural_ApplicantIncome',
 'Property_Area_Rural_LoanAmount',
 'Property_Area_Semiurban_Credit_History',
 'Property_Area_Semiurban_Loan_Amount_Term',
 'Property_Area_Urban_Credit_History',
 'ApplicantIncome_Credit_History',
 'CoapplicantIncome_Credit_History',
 'Credit_History_LoanAmount',
 'Credit_History_Loan_Amount_Term']

In [2]:
from sklearn.linear_model import Ridge

In [3]:
from xgboost import XGBClassifier

In [ ]:
XGBClassifier()

In [7]:
from sklearn.linear_model import LogisticRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = LogisticRegression()
modelfit(alg1, train, test, predictors, target, IDcol, 'res2.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#print(coef1)
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
             precision    recall  f1-score   support

          N       0.93      0.50      0.65       192
          Y       0.81      0.98      0.89       422

avg / total       0.85      0.83      0.81       614



In [71]:
from sklearn.neighbors import KNeighborsClassifier
for i in range(1,20):
    alg1 = KNeighborsClassifier(n_neighbors=i)
    print(i)
    modelfit(alg1, train, test, predictors, target, IDcol, 'res6.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

1

Model Report
             precision    recall  f1-score   support

          N       0.98      0.99      0.99       192
          Y       1.00      0.99      1.00       422

avg / total       0.99      0.99      0.99       614

2

Model Report
             precision    recall  f1-score   support

          N       0.73      1.00      0.84       192
          Y       1.00      0.83      0.91       422

avg / total       0.92      0.88      0.89       614

3

Model Report
             precision    recall  f1-score   support

          N       0.86      0.69      0.77       192
          Y       0.87      0.95      0.91       422

avg / total       0.87      0.87      0.87       614

4

Model Report
             precision    recall  f1-score   support

          N       0.78      0.73      0.76       192
          Y       0.88      0.91      0.89       422

avg / total       0.85      0.85      0.85       614

5

Model Report
             precision    recall  f1-score   support

      

In [65]:
alg1 = KNeighborsClassifier(n_neighbors=17)
modelfit(alg1, train, test, predictors, target, IDcol, 'res6.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
             precision    recall  f1-score   support

          N       0.92      0.51      0.65       192
          Y       0.81      0.98      0.89       422

avg / total       0.85      0.83      0.82       614



In [2]:
from sklearn.tree import DecisionTreeClassifier

In [74]:

#DecisionTreeRegressor
#predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = DecisionTreeClassifier(max_depth=5)#DecisionTreeRegressor(max_depth=5, min_samples_split=2,min_samples_leaf=1)
modelfit(alg1, train, test, predictors, target, IDcol, 'res7.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
             precision    recall  f1-score   support

          N       0.94      0.61      0.74       192
          Y       0.85      0.98      0.91       422

avg / total       0.87      0.86      0.86       614



In [1]:
from sklearn.ensemble import RandomForestClassifier

In [85]:
#predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = RandomForestClassifier(n_estimators=250,max_depth=6, random_state=1)
#alg1 = RandomForestRegressor(n_estimators=80,max_depth=15)
#alg1 = RandomForestRegressor(n_estimators=500,max_depth=10, min_samples_split=100,min_samples_leaf=50)
modelfit(alg1, train, test, predictors, target, IDcol, 'res8.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
             precision    recall  f1-score   support

          N       0.94      0.56      0.70       192
          Y       0.83      0.98      0.90       422

avg / total       0.87      0.85      0.84       614



In [3]:
from sklearn.ensemble import AdaBoostClassifier
AdaBoostClassifier()

In [9]:
train.shape

(614, 22)

In [14]:
train.head()

,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,...,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_ID,Loan_Status,ApplicantIncome,CoapplicantIncome,Credit_History,LoanAmount,Loan_Amount_Term
0,0,1,1,0,1,0,0,0,1,0,...,0,0,1,LP001002,Y,5849.0,0.0,1.0,133.0,360.0
1,0,1,0,1,0,1,0,0,1,0,...,1,0,0,LP001003,N,4583.0,1508.0,1.0,128.0,360.0
2,0,1,0,1,1,0,0,0,1,0,...,0,0,1,LP001005,Y,3000.0,0.0,1.0,66.0,360.0
3,0,1,0,1,1,0,0,0,0,1,...,0,0,1,LP001006,Y,2583.0,2358.0,1.0,120.0,360.0
4,0,1,1,0,1,0,0,0,1,0,...,0,0,1,LP001008,Y,6000.0,0.0,1.0,141.0,360.0


In [10]:
test.shape

(367, 21)

In [33]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [122]:
target = 'Loan_Status'
IDcol = ['Loan_ID']
train['Loan_Status']=train.Loan_Status.replace({'Y':1,'N':0})
predictors = [x for x in train.columns if x not in [target]+IDcol]


In [123]:
train[predictors].head()

,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,...,ApplicantIncome_CoapplicantIncome,ApplicantIncome_Credit_History,ApplicantIncome_LoanAmount,ApplicantIncome_Loan_Amount_Term,CoapplicantIncome_Credit_History,CoapplicantIncome_LoanAmount,CoapplicantIncome_Loan_Amount_Term,Credit_History_LoanAmount,Credit_History_Loan_Amount_Term,LoanAmount_Loan_Amount_Term
0,0,1,1,0,1,0,0,0,1,0,...,0.0,5849.0,777917.0,2105640.0,0.0,0.0,0.0,133.0,360.0,47880.0
1,0,1,0,1,0,1,0,0,1,0,...,6911164.0,4583.0,586624.0,1649880.0,1508.0,193024.0,542880.0,128.0,360.0,46080.0
2,0,1,0,1,1,0,0,0,1,0,...,0.0,3000.0,198000.0,1080000.0,0.0,0.0,0.0,66.0,360.0,23760.0
3,0,1,0,1,1,0,0,0,0,1,...,6090714.0,2583.0,309960.0,929880.0,2358.0,282960.0,848880.0,120.0,360.0,43200.0
4,0,1,1,0,1,0,0,0,1,0,...,0.0,6000.0,846000.0,2160000.0,0.0,0.0,0.0,141.0,360.0,50760.0


In [212]:
train.Loan_Status.unique()

array([1, 0], dtype=int64)

In [209]:
#Define target and ID columns:

from sklearn import model_selection, metrics
from sklearn.metrics import classification_report
from keras import regularizers

def modelfit(alg, dtrain, dtest, predictors, target, IDcol,filename):
    #Fit the algorithm on the data
    

    #for val in range(len(ncount)):
    #    model.add(Dense(ncount[val], kernel_initializer = 'uniform',activation='elu'))
    #model.add(Dense(1, kernel_initializer = 'uniform',activation='sigmoid'))
    
    alg.add(Dense(197, input_dim=197,kernel_initializer = 'uniform', activation='elu'))
    alg.add(Dropout(0.7))
    alg.add(Dense(500, kernel_initializer = 'uniform',activation='elu'))
    alg.add(Dropout(0.7))
    '''
    alg.add(Dense(400, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.7))
    alg.add(Dense(300, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.5))
  
    alg.add(Dense(1000, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.5))
    alg.add(Dense(1000, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.5))
    alg.add(Dense(1000, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.5))
    alg.add(Dense(1000, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.2))
    '''
    alg.add(Dense(1000, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dropout(0.1))
    alg.add(Dense(500, kernel_initializer = 'uniform',activation='relu'))
    alg.add(Dense(2, kernel_initializer = 'uniform',activation='sigmoid'))
    
    alg.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    tensorboard = TensorBoard(log_dir='tensor_simple', histogram_freq=0,write_graph=True, write_images=True)
    alg.fit(dtrain[predictors], dtrain[target], epochs=500,shuffle=True,callbacks=[tensorboard])
    scores = alg.evaluate(dtrain[predictors], dtrain[target])
    print("\n%s: %.2f%%" % (alg.metrics_names[1], scores[1]*100))
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors],batch_size=10)
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission['Loan_Status']=submission.Loan_Status.replace({1.0:'Y',0.0:'N'})
    print(submission.head())
    submission.to_csv(filename, index=False)

In [210]:
alg1 = Sequential()

In [211]:
modelfit(alg1, train, test, predictors, target, IDcol,'DL.csv')

ValueError: Error when checking target: expected dense_304 to have shape (2,) but got array with shape (1,)

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='tensor_simple', histogram_freq=0,
                          write_graph=True, write_images=True)

In [14]:
X.shape

(614, 20)

In [15]:
Y.shape

(614,)

In [19]:
model.fit(X_train, Y_train, epochs=500, batch_size=50,shuffle=True,callbacks=[tensorboard])
scores = model.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/500
614/614 [==============================] - 0s 43us/step - loss: 0.4535 - acc: 0.7785
Epoch 2/500
614/614 [==============================] - 0s 29us/step - loss: 0.4655 - acc: 0.7687
Epoch 3/500
614/614 [==============================] - 0s 35us/step - loss: 0.4504 - acc: 0.7850
Epoch 4/500
614/614 [==============================] - 0s 38us/step - loss: 0.4366 - acc: 0.7932
Epoch 5/500
614/614 [==============================] - 0s 32us/step - loss: 0.4429 - acc: 0.7801
Epoch 6/500
614/614 [==============================] - 0s 33us/step - loss: 0.4775 - acc: 0.7671
Epoch 7/500
614/614 [==============================] - 0s 35us/step - loss: 0.4656 - acc: 0.7720
Epoch 8/500
614/614 [==============================] - ETA: 0s - loss: 0.4937 - acc: 0.780 - 0s 32us/step - loss: 0.4514 - acc: 0.7818
Epoch 9/500
614/614 [==============================] - 0s 40us/step - loss: 0.4324 - acc: 0.7834
Epoch 10/500
614/614 [==============================] - 0s 28us/step - loss: 0.4737 - acc

614/614 [==============================] - 0s 28us/step - loss: 0.3607 - acc: 0.8192
Epoch 166/500
614/614 [==============================] - 0s 28us/step - loss: 0.3478 - acc: 0.8306
Epoch 167/500
614/614 [==============================] - 0s 28us/step - loss: 0.3611 - acc: 0.8388
Epoch 168/500
614/614 [==============================] - 0s 32us/step - loss: 0.3474 - acc: 0.8371
Epoch 169/500
614/614 [==============================] - 0s 30us/step - loss: 0.3584 - acc: 0.8306
Epoch 170/500
614/614 [==============================] - 0s 26us/step - loss: 0.3739 - acc: 0.8257
Epoch 171/500
614/614 [==============================] - 0s 30us/step - loss: 0.3489 - acc: 0.8371
Epoch 172/500
614/614 [==============================] - 0s 31us/step - loss: 0.4022 - acc: 0.7948
Epoch 173/500
614/614 [==============================] - 0s 30us/step - loss: 0.3971 - acc: 0.8176
Epoch 174/500
614/614 [==============================] - 0s 31us/step - loss: 0.3664 - acc: 0.8339
Epoch 175/500
614/614 [=

614/614 [==============================] - 0s 28us/step - loss: 0.3316 - acc: 0.8534
Epoch 248/500
614/614 [==============================] - 0s 30us/step - loss: 0.3087 - acc: 0.8599
Epoch 249/500
614/614 [==============================] - 0s 30us/step - loss: 0.3364 - acc: 0.8453
Epoch 250/500
614/614 [==============================] - 0s 35us/step - loss: 0.3309 - acc: 0.8485
Epoch 251/500
614/614 [==============================] - 0s 34us/step - loss: 0.3989 - acc: 0.8290
Epoch 252/500
614/614 [==============================] - 0s 31us/step - loss: 0.3409 - acc: 0.8257
Epoch 253/500
614/614 [==============================] - 0s 33us/step - loss: 0.3434 - acc: 0.8485
Epoch 254/500
614/614 [==============================] - 0s 35us/step - loss: 0.3186 - acc: 0.8485
Epoch 255/500
614/614 [==============================] - 0s 31us/step - loss: 0.3462 - acc: 0.8436
Epoch 256/500
614/614 [==============================] - 0s 34us/step - loss: 0.3276 - acc: 0.8567
Epoch 257/500
614/614 [=

Epoch 328/500
614/614 [==============================] - 0s 29us/step - loss: 0.3291 - acc: 0.8502
Epoch 329/500
614/614 [==============================] - 0s 30us/step - loss: 0.3095 - acc: 0.8664
Epoch 330/500
614/614 [==============================] - 0s 31us/step - loss: 0.3246 - acc: 0.8550
Epoch 331/500
614/614 [==============================] - 0s 43us/step - loss: 0.4087 - acc: 0.8094
Epoch 332/500
614/614 [==============================] - ETA: 0s - loss: 0.3465 - acc: 0.880 - 0s 33us/step - loss: 0.3950 - acc: 0.8176
Epoch 333/500
614/614 [==============================] - 0s 33us/step - loss: 0.3815 - acc: 0.8290
Epoch 334/500
614/614 [==============================] - 0s 30us/step - loss: 0.3166 - acc: 0.8616
Epoch 335/500
614/614 [==============================] - 0s 32us/step - loss: 0.3446 - acc: 0.8550
Epoch 336/500
614/614 [==============================] - ETA: 0s - loss: 0.2814 - acc: 0.880 - 0s 31us/step - loss: 0.3617 - acc: 0.8322
Epoch 337/500
614/614 [==========

614/614 [==============================] - 0s 26us/step - loss: 0.2885 - acc: 0.8827
Epoch 490/500
614/614 [==============================] - 0s 22us/step - loss: 0.3089 - acc: 0.8616
Epoch 491/500
614/614 [==============================] - 0s 22us/step - loss: 0.2729 - acc: 0.8795
Epoch 492/500
614/614 [==============================] - 0s 22us/step - loss: 0.2640 - acc: 0.8844
Epoch 493/500
614/614 [==============================] - 0s 32us/step - loss: 0.2743 - acc: 0.8844
Epoch 494/500
614/614 [==============================] - 0s 22us/step - loss: 0.2693 - acc: 0.8827
Epoch 495/500
614/614 [==============================] - 0s 21us/step - loss: 0.2684 - acc: 0.8844
Epoch 496/500
614/614 [==============================] - ETA: 0s - loss: 0.2751 - acc: 0.880 - 0s 23us/step - loss: 0.2580 - acc: 0.8925
Epoch 497/500
614/614 [==============================] - 0s 23us/step - loss: 0.2589 - acc: 0.8941
Epoch 498/500
614/614 [==============================] - 0s 25us/step - loss: 0.2441 

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                1050      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
_________________________________________________________________
dense_5 (Dense)              (None, 50)                100       
_________________________________________________________________
dense_6 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                310       
__________

In [ ]:
model.predict()